In [1]:
from pyomo.environ import *
import mpisppy.utils.sputils as sputils
import matplotlib.pyplot as plt
from matplotlib import rc
import sys
import itertools
sys.path.append('../../../../src')
import pandas
import random
import math
from energiapy.components.temporal_scale import TemporalScale
from energiapy.components.resource import Resource, VaryingResource
from energiapy.components.process import Process, ProcessMode, VaryingProcess
from energiapy.components.location import Location
from energiapy.components.transport import Transport, VaryingTransport
from energiapy.components.network import Network
from energiapy.components.scenario import Scenario
# from energiapy.model.constraints.demand import constraint_demand2
from energiapy.components.result import Result
from energiapy.model.formulate import formulate, Constraints, Objective
from energiapy.plot import plot_results, plot_scenario, plot_location
from energiapy.model.solve import solve
from pyomo.environ import Param
from energiapy.utils.scale_utils import scale_pyomo_set
from energiapy.utils.scale_utils import scale_list, scale_tuple

[    0.00] Initializing mpi-sppy


In [2]:
_time_intervals = 7  # Number of time intervals in a planning horizon    (L_chi)
_coms = 1
_exec_scenarios = 56  # Number of execution scenarios                     (chi)

M = 1e4  # Big M

In [3]:
n_dict = {1: 14, 2: 28, 3: 42, 4: 56}

def create_list(n:int):
    return [1]*n_dict[n] + [0]*(_exec_scenarios - n_dict[n])

scenario_dictionary = {
    'cap1_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1','com1_process'): create_list(1)})},
    'cap1_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(2)})},
    'cap1_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(3)})},
    'cap1_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_process'): create_list(4)})},
             
    'cap2_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2','com1_process'): create_list(1)})},
    'cap2_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(2)})},
    'cap2_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(3)})},
    'cap2_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_process'): create_list(4)})},
     
    'cap3_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3','com1_process'): create_list(1)})},
    'cap3_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(2)})},
    'cap3_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(3)})},
    'cap3_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_process'): create_list(4)})},
     
    # 'cap4_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc4','com1_process'): create_list(1)})},
    # 'cap4_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(2)})},
    # 'cap4_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(3)})},
    # 'cap4_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc4', 'com1_process'): create_list(4)})},
     
    'cap5_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5','com1_process'): create_list(1)})},
    'cap5_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(2)})},
    'cap5_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(3)})},
    'cap5_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc5', 'com1_process'): create_list(4)})},
     
    'cap6_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6','com1_process'): create_list(1)})},
    'cap6_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(2)})},
    'cap6_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(3)})},
    'cap6_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc6', 'com1_process'): create_list(4)})},
     
    'cap7_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7','com1_process'): create_list(1)})},
    'cap7_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(2)})},
    'cap7_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(3)})},
    'cap7_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc7', 'com1_process'): create_list(4)})},
     
    # 'cap8_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc8','com1_process'): create_list(1)})},
    # 'cap8_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc8', 'com1_process'): create_list(2)})},
    # 'cap8_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc8', 'com1_process'): create_list(3)})},
    # 'cap8_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc8', 'com1_process'): create_list(4)})},
    #  
    # 'cap9_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc9','com1_process'): create_list(1)})},
    # 'cap9_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc9', 'com1_process'): create_list(2)})},
    # 'cap9_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc9', 'com1_process'): create_list(3)})},
    # 'cap9_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc9', 'com1_process'): create_list(4)})},
     
    'cap10_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc10','com1_process'): create_list(1)})},
    'cap10_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc10', 'com1_process'): create_list(2)})},
    'cap10_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc10', 'com1_process'): create_list(3)})},
    'cap10_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc10', 'com1_process'): create_list(4)})},
     
    # 'cap11_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc11','com1_process'): create_list(1)})},
    # 'cap11_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc11', 'com1_process'): create_list(2)})},
    # 'cap11_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc11', 'com1_process'): create_list(3)})},
    # 'cap11_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc11', 'com1_process'): create_list(4)})},
    #  
    # 'cap12_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc12','com1_process'): create_list(1)})},
    # 'cap12_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc12', 'com1_process'): create_list(2)})},
    # 'cap12_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc12', 'com1_process'): create_list(3)})},
    # 'cap12_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc12', 'com1_process'): create_list(4)})},
    #  
    # 'cap13_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc13','com1_process'): create_list(1)})},
    # 'cap13_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc13', 'com1_process'): create_list(2)})},
    # 'cap13_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc13', 'com1_process'): create_list(3)})},
    # 'cap13_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc13', 'com1_process'): create_list(4)})},
     
    'res1_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1','com1_pur'): create_list(1)})},
    'res1_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(2)})},
    'res1_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(3)})},
    'res1_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc1', 'com1_pur'): create_list(4)})},
     
    'res2_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2','com1_pur'): create_list(1)})},
    'res2_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_pur'): create_list(2)})},
    'res2_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_pur'): create_list(3)})},
    'res2_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc2', 'com1_pur'): create_list(4)})},
     
    'res3_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3','com1_pur'): create_list(1)})},
    'res3_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_pur'): create_list(2)})},
    'res3_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_pur'): create_list(3)})},
    'res3_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={('loc3', 'com1_pur'): create_list(4)})},
     
    # 'trans14_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(1)})},
    # 'trans14_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(2)})},
    # 'trans14_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(3)})},
    # 'trans14_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans14': create_list(4)})},
    #  
    # 'trans15_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans15': create_list(1)})},
    # 'trans15_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans15': create_list(2)})},
    # 'trans15_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans15': create_list(3)})},
    # 'trans15_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans15': create_list(4)})},
    #  
    # 'trans16_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans16': create_list(1)})},
    # 'trans16_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans16': create_list(2)})},
    # 'trans16_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans16': create_list(3)})},
    # 'trans16_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans16': create_list(4)})},
    #  
    # 'trans25_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans25': create_list(1)})},
    # 'trans25_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans25': create_list(2)})},
    # 'trans25_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans25': create_list(3)})},
    # 'trans25_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans25': create_list(4)})},
    #  
    # 'trans26_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans26': create_list(1)})},
    # 'trans26_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans26': create_list(2)})},
    # 'trans26_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans26': create_list(3)})},
    # 'trans26_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans26': create_list(4)})},
    #  
    # 'trans27_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans27': create_list(1)})},
    # 'trans27_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans27': create_list(2)})},
    # 'trans27_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans27': create_list(3)})},
    # 'trans27_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans27': create_list(4)})},
    #  
    # 'trans36_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans36': create_list(1)})},
    # 'trans36_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans36': create_list(2)})},
    # 'trans36_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans36': create_list(3)})},
    # 'trans36_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans36': create_list(4)})},
    #  
    # 'trans37_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans37': create_list(1)})},
    # 'trans37_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans37': create_list(2)})},
    # 'trans37_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans37': create_list(3)})},
    # 'trans37_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans37': create_list(4)})},
    #  
    # 'trans38_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans38': create_list(1)})},
    # 'trans38_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans38': create_list(2)})},
    # 'trans38_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans38': create_list(3)})},
    # 'trans38_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans38': create_list(4)})},
    #  
    # 'trans49_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans49': create_list(1)})},
    # 'trans49_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans49': create_list(2)})},
    # 'trans49_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans49': create_list(3)})},
    # 'trans49_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans49': create_list(4)})},
    #  
    # 'trans59_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans59': create_list(1)})},
    # 'trans59_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans59': create_list(2)})},
    # 'trans59_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans59': create_list(3)})},
    # 'trans59_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans59': create_list(4)})},
    #  
    # 'trans69_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans69': create_list(1)})},
    # 'trans69_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans69': create_list(2)})},
    # 'trans69_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans69': create_list(3)})},
    # 'trans69_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans69': create_list(4)})},
    #  
    # 'trans710_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans710': create_list(1)})},
    # 'trans710_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans710': create_list(2)})},
    # 'trans710_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans710': create_list(3)})},
    # 'trans710_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans710': create_list(4)})},
    #  
    # 'trans711_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans711': create_list(1)})},
    # 'trans711_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans711': create_list(2)})},
    # 'trans711_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans711': create_list(3)})},
    # 'trans711_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans711': create_list(4)})},
    #  
    # 'trans811_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans811': create_list(1)})},
    # 'trans811_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans811': create_list(2)})},
    # 'trans811_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans811': create_list(3)})},
    # 'trans811_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans811': create_list(4)})},
    #  
    # 'trans912_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans912': create_list(1)})},
    # 'trans912_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans912': create_list(2)})},
    # 'trans912_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans912': create_list(3)})},
    # 'trans912_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans912': create_list(4)})},
    #  
    # 'trans1012_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1012': create_list(1)})},
    # 'trans1012_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1012': create_list(2)})},
    # 'trans1012_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1012': create_list(3)})},
    # 'trans1012_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1012': create_list(4)})},
    #  
    # 'trans1013_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1013': create_list(1)})},
    # 'trans1013_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1013': create_list(2)})},
    # 'trans1013_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1013': create_list(3)})},
    # 'trans1013_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1013': create_list(4)})},
    #  
    # 'trans1113_1': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1113': create_list(1)})},
    # 'trans1113_2': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1113': create_list(2)})},
    # 'trans1113_3': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1113': create_list(3)})},
    # 'trans1113_4': {'prob': 0.25, 'factor': pandas.DataFrame(data={'trans1113': create_list(4)})}
}

scenario_names = list(scenario_dictionary.keys())

In [5]:
scenario_names

40

In [ ]:
def build_model(scen_dict, scen_name):
    
    default_df = pandas.DataFrame(data=[1]*_exec_scenarios)
    
    # Define temporal scales
    scales = TemporalScale(discretization_list=[1, _exec_scenarios, _time_intervals])
    
    # ======================================================================================================================
    # Declare resources/commodities
    # ======================================================================================================================
    com1_pur = Resource(name='com1_pur', cons_max=M, block={'imp': 1, 'urg': 1}, price=0.00, label='Commodity 1 consumed from outside the system', varying=[VaryingResource.DETERMINISTIC_AVAILABILITY])
    
    com1_in = Resource(name='com1_in', label='Commodity 1 received')
    com1_out = Resource(name='com1_out', label='Commodity 1 to be sent out')
    
    com1_loc1_out = Resource(name='com1_loc1_out', label='Commodity 1 sent out from location 1')
    com1_loc2_out = Resource(name='com1_loc2_out', label='Commodity 1 sent out from location 2')
    com1_loc3_out = Resource(name='com1_loc3_out', label='Commodity 1 sent out from location 3')
    com1_loc4_out = Resource(name='com1_loc4_out', label='Commodity 1 sent out from location 4')
    com1_loc5_out = Resource(name='com1_loc5_out', label='Commodity 1 sent out from location 5')
    com1_loc6_out = Resource(name='com1_loc6_out', label='Commodity 1 sent out from location 6')
    com1_loc7_out = Resource(name='com1_loc7_out', label='Commodity 1 sent out from location 7')
    com1_loc8_out = Resource(name='com1_loc8_out', label='Commodity 1 sent out from location 8')
    com1_loc9_out = Resource(name='com1_loc9_out', label='Commodity 1 sent out from location 9')
    com1_loc10_out = Resource(name='com1_loc10_out', label='Commodity 1 sent out from location 10')
    com1_loc11_out = Resource(name='com1_loc11_out', label='Commodity 1 sent out from location 11')
    com1_loc12_out = Resource(name='com1_loc12_out', label='Commodity 1 sent out from location 12')
    com1_loc13_out = Resource(name='com1_loc13_out', label='Commodity 1 sent out from location 13')
    
    com1_sold = Resource(name='com1_sold', revenue=0.00, demand=True, sell=True, label='Commodity 1 sold to outside the system')
    
    
    # ======================================================================================================================
    # Declare processes/storage capacities
    # ======================================================================================================================
    com1_process_capacity = 250
    
    # prod_max = {0: 0.25*com1_process_capacity, 1: 0.5*com1_process_capacity, 2: 0.75*com1_process_capacity, 3: 0.95*com1_process_capacity, 4: com1_process_capacity}
    # prod_min = {0: 0, 1: 0.25*com1_process_capacity, 2: 0.5*com1_process_capacity, 3: 0.75*com1_process_capacity, 4: 0.95*com1_process_capacity}
    # rate_max = {0:1.25/2, 1: 1/2, 2: 0.75/2, 3: 0.5/2, 4: 0.25/2}
    # mode_ramp = {(0,1): 5, (1,2): 5}
    
    com1_procure = Process(name='procure com1', prod_max=com1_process_capacity, conversion={com1_pur: -1, com1_in: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Procure com1')
    com1_sell = Process(name='sell com1', prod_max=com1_process_capacity, conversion={com1_out: -1, com1_sold: 1}, capex=0.1, vopex=0.01, prod_min=0.01, label='Sell com1')
    
    vopex_t1 = 0.01
    vopex_t2_low = 0.02
    vopex_t2_mid = 0.05
    vopex_t2_high = 0.08
    vopex_t3_low = 0.02
    vopex_t3_high = 0.04
    vopex_t4 = 0.01
    
    # capex_t1 = 0.01
    # capex_t2_low = 0.02
    # capex_t2_mid = 0.05
    # capex_t2_high = 0.08
    # capex_t3_low = 0.02
    # capex_t3_high = 0.04
    # capex_t4 = 0.01
    
    com1_receive_loc1 = Process(name='com1_receive_loc1', prod_max=com1_process_capacity, conversion={com1_loc1_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 1')
    com1_receive_loc2 = Process(name='com1_receive_loc2', prod_max=com1_process_capacity, conversion={com1_loc2_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 2')
    com1_receive_loc3 = Process(name='com1_receive_loc3', prod_max=com1_process_capacity, conversion={com1_loc3_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Commodity 1 received from location 3')
    com1_receive_loc4 = Process(name='com1_receive_loc4', prod_max=com1_process_capacity, conversion={com1_loc4_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Commodity 1 received from location 4')
    com1_receive_loc5 = Process(name='com1_receive_loc5', prod_max=com1_process_capacity, conversion={com1_loc5_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Commodity 1 received from location 5')
    com1_receive_loc6 = Process(name='com1_receive_loc6', prod_max=com1_process_capacity, conversion={com1_loc6_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_low, prod_min=0.01, label='Commodity 1 received from location 6')
    com1_receive_loc7 = Process(name='com1_receive_loc7', prod_max=com1_process_capacity, conversion={com1_loc7_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Commodity 1 received from location 7')
    com1_receive_loc8 = Process(name='com1_receive_loc8', prod_max=com1_process_capacity, conversion={com1_loc8_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Commodity 1 received from location 8')
    com1_receive_loc9 = Process(name='com1_receive_loc9', prod_max=com1_process_capacity, conversion={com1_loc9_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Commodity 1 received from location 9')
    com1_receive_loc10 = Process(name='com1_receive_loc10', prod_max=com1_process_capacity, conversion={com1_loc10_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_low, prod_min=0.01, label='Commodity 1 received from location 10')
    com1_receive_loc11 = Process(name='com1_receive_loc11', prod_max=com1_process_capacity, conversion={com1_loc11_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Commodity 1 received from location 11')
    com1_receive_loc12 = Process(name='com1_receive_loc12', prod_max=com1_process_capacity, conversion={com1_loc12_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Commodity 1 received from location 12')
    com1_receive_loc13 = Process(name='com1_receive_loc13', prod_max=com1_process_capacity, conversion={com1_loc13_out:-1, com1_in:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Commodity 1 received from location 13')
    
    com1_process = Process(name='com1_process', prod_max=com1_process_capacity, conversion={com1_in: -1, com1_out: 1},  capex=0.1, vopex=0.01, prod_min=0.01, label='Process the commodity through the location', varying=[VaryingProcess.DETERMINISTIC_CAPACITY])
    
    com1_store = Process(name='com1_store', prod_max=com1_process_capacity, capex=0.1, vopex=0.01, storage_capex=10, store_min=0.01, store_max= M, prod_min=0.01, label="Storage process", storage=com1_in, storage_cost=0.02)
    
    com1_loc1_send = Process(name='com1_loc1_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc1_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 1')
    com1_loc2_send = Process(name='com1_loc2_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc2_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 2')
    com1_loc3_send = Process(name='com1_loc3_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc3_out:1}, capex=0.1, vopex=vopex_t1, prod_min=0.01, label='Send commodity one from location 3')
    com1_loc4_send = Process(name='com1_loc4_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc4_out:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Send commodity one from location 4')
    com1_loc5_send = Process(name='com1_loc5_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc5_out:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Send commodity one from location 5')
    com1_loc6_send = Process(name='com1_loc6_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc6_out:1}, capex=0.1, vopex=vopex_t2_low, prod_min=0.01, label='Send commodity one from location 6')
    com1_loc7_send = Process(name='com1_loc7_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc7_out:1}, capex=0.1, vopex=vopex_t2_mid, prod_min=0.01, label='Send commodity one from location 7')
    com1_loc8_send = Process(name='com1_loc8_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc8_out:1}, capex=0.1, vopex=vopex_t2_high, prod_min=0.01, label='Send commodity one from location 8')
    com1_loc9_send = Process(name='com1_loc9_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc9_out:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Send commodity one from location 9')
    com1_loc10_send = Process(name='com1_loc10_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc10_out:1}, capex=0.1, vopex=vopex_t3_low, prod_min=0.01, label='Send commodity one from location 10')
    com1_loc11_send = Process(name='com1_loc11_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc11_out:1}, capex=0.1, vopex=vopex_t3_high, prod_min=0.01, label='Send commodity one from location 11')
    com1_loc12_send = Process(name='com1_loc12_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc12_out:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Send commodity one from location 12')
    com1_loc13_send = Process(name='com1_loc13_send', prod_max=com1_process_capacity, conversion={com1_out:-1, com1_loc13_out:1}, capex=0.1, vopex=vopex_t4, prod_min=0.01, label='Send commodity one from location 13')
    
    # ======================================================================================================================
    # Declare locations/warehouses
    # ======================================================================================================================
    loc1 = Location(name='loc1', processes={com1_procure, com1_receive_loc4, com1_receive_loc5, com1_receive_loc6, com1_process, com1_store, com1_loc1_send}, label="Location 1", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1, 
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc1','com1_pur')]] if ('loc1', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc1', 'com1_process')]] if ('loc1', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc2 = Location(name='loc2', processes={com1_procure, com1_receive_loc5, com1_receive_loc6, com1_receive_loc7, com1_process, com1_store, com1_loc2_send}, label="Location 2", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc2','com1_pur')]] if ('loc2', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc2', 'com1_process')]] if ('loc2', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc3 = Location(name='loc3', processes={com1_procure, com1_receive_loc6, com1_receive_loc7, com1_receive_loc8, com1_process, com1_store, com1_loc3_send}, label="Location 3", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc3','com1_pur')]] if ('loc3', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc3', 'com1_process')]] if ('loc3', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc4 = Location(name='loc4', processes={com1_receive_loc1, com1_receive_loc9, com1_process, com1_store, com1_loc4_send}, label="Location 4", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc4','com1_pur')]] if ('loc4', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc4', 'com1_process')]] if ('loc4', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc5 = Location(name='loc5', processes={com1_receive_loc1, com1_receive_loc2, com1_receive_loc9, com1_process, com1_store, com1_loc5_send}, label="Location 5", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc5','com1_pur')]] if ('loc5', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc5', 'com1_process')]] if ('loc5', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc6 = Location(name='loc6', processes={com1_receive_loc1, com1_receive_loc2, com1_receive_loc3, com1_receive_loc9, com1_receive_loc10, com1_process, com1_store, com1_loc6_send}, label="Location 6", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc6','com1_pur')]] if ('loc6', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc6', 'com1_process')]] if ('loc6', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc7 = Location(name='loc7', processes={com1_receive_loc2, com1_receive_loc3, com1_receive_loc10, com1_receive_loc11, com1_process, com1_store, com1_loc7_send}, label="Location 7", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc7','com1_pur')]] if ('loc7', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc7', 'com1_process')]] if ('loc7', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc8 = Location(name='loc8', processes={com1_receive_loc3, com1_receive_loc11, com1_process, com1_store, com1_loc8_send}, label="Location 8", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc8','com1_pur')]] if ('loc8', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc8', 'com1_process')]] if ('loc8', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc9 = Location(name='loc9', processes={com1_receive_loc4, com1_receive_loc5, com1_receive_loc6, com1_receive_loc12, com1_process, com1_store, com1_loc9_send}, label="Location 9", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc9','com1_pur')]] if ('loc9', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc9', 'com1_process')]] if ('loc9', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc10 = Location(name='loc10', processes={com1_receive_loc6, com1_receive_loc7, com1_receive_loc12, com1_receive_loc13, com1_process, com1_store, com1_loc10_send}, label="Location 10", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc10','com1_pur')]] if ('loc10', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc10', 'com1_process')]] if ('loc10', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc11 = Location(name='loc11', processes={com1_receive_loc7, com1_receive_loc8, com1_receive_loc13, com1_process, com1_store, com1_loc11_send}, label="Location 9", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc11','com1_pur')]] if ('loc11', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc11', 'com1_process')]] if ('loc11', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc12 = Location(name='loc12', processes={com1_receive_loc9, com1_receive_loc10, com1_process, com1_store, com1_loc12_send, com1_sell}, label="Location 12", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc12','com1_pur')]] if ('loc12', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc12', 'com1_process')]] if ('loc12', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    loc13 = Location(name='loc13', processes={com1_receive_loc10, com1_receive_loc11, com1_process, com1_store, com1_loc13_send, com1_sell}, label="Location 13", scales=scales, demand_scale_level=2, capacity_scale_level=1, availability_scale_level=1,
            availability_factor={com1_pur: scen_dict[scen_name]['factor'][[('loc13','com1_pur')]] if ('loc13', 'com1_pur') in scen_dict[scen_name]['factor'] else default_df},
            capacity_factor={com1_process: scen_dict[scen_name]['factor'][[('loc13', 'com1_process')]] if ('loc13', 'com1_process') in scen_dict[scen_name]['factor'] else default_df})
    
    
    # ======================================================================================================================
    # Declare transport/trucks
    # ======================================================================================================================
    
    truck_large = 200
    truck_medium = 125
    truck_small = 75
    
    truck14 = Transport(name='truck14', resources={com1_loc1_out}, trans_max=truck_small, label='Truck from location 1 to 4', capex=0.5, vopex=0.05, trans_min=truck_small)
    truck41 = Transport(name='truck41', resources={com1_loc4_out}, trans_max=truck_small, label='Truck from location 4 to 1', capex=0.0001, vopex=0.05, trans_min=truck_small)
    
    truck15 = Transport(name='truck15', resources={com1_loc1_out}, trans_max=truck_medium, label='Truck from location 1 to 5', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck51 = Transport(name='truck51', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 1', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck16 = Transport(name='truck16', resources={com1_loc1_out}, trans_max=truck_medium, label='Truck from location 1 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck61 = Transport(name='truck61', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 1', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck25 = Transport(name='truck25', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 5', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck52 = Transport(name='truck52', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck26 = Transport(name='truck26', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck62 = Transport(name='truck62', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck27 = Transport(name='truck27', resources={com1_loc2_out}, trans_max=truck_medium, label='Truck from location 2 to 7', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck72 = Transport(name='truck72', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 2', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck36 = Transport(name='truck36', resources={com1_loc3_out}, trans_max=truck_medium, label='Truck from location 3 to 6', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck63 = Transport(name='truck63', resources={com1_loc6_out}, trans_max=truck_medium, label='Truck from location 6 to 3', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck37 = Transport(name='truck37', resources={com1_loc3_out}, trans_max=truck_medium, label='Truck from location 3 to 7', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck73 = Transport(name='truck73', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 3', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck38 = Transport(name='truck38', resources={com1_loc3_out}, trans_max=truck_small, label='Truck from location 3 to 8', capex=0.5, vopex=0.05, trans_min=truck_small)
    truck83 = Transport(name='truck83', resources={com1_loc8_out}, trans_max=truck_small, label='Truck from location 8 to 3', capex=0.0001, vopex=0.05, trans_min=truck_small)
    
    truck49 = Transport(name='truck49', resources={com1_loc4_out}, trans_max=truck_medium, label='Truck from location 4 to 9', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck94 = Transport(name='truck94', resources={com1_loc9_out}, trans_max=truck_medium, label='Truck from location 9 to 4', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck59 = Transport(name='truck59', resources={com1_loc5_out}, trans_max=truck_medium, label='Truck from location 5 to 9', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck95 = Transport(name='truck95', resources={com1_loc9_out}, trans_max=truck_medium, label='Truck from location 9 to 5', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck69 = Transport(name='truck69', resources={com1_loc6_out}, trans_max=truck_large, label='Truck from location 6 to 9', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck96 = Transport(name='truck96', resources={com1_loc9_out}, trans_max=truck_large, label='Truck from location 9 to 6', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck610 = Transport(name='truck610', resources={com1_loc6_out}, trans_max=truck_large, label='Truck from location 6 to 10', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck106 = Transport(name='truck106', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 6', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck710 = Transport(name='truck710', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 10', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck107 = Transport(name='truck107', resources={com1_loc10_out}, trans_max=truck_medium, label='Truck from location 10 to 7', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck711 = Transport(name='truck711', resources={com1_loc7_out}, trans_max=truck_medium, label='Truck from location 7 to 11', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck117 = Transport(name='truck117', resources={com1_loc11_out}, trans_max=truck_medium, label='Truck from location 11 to 7', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck811 = Transport(name='truck811', resources={com1_loc8_out}, trans_max=truck_medium, label='Truck from location 8 to 11', capex=0.5, vopex=0.05, trans_min=truck_medium)
    truck118 = Transport(name='truck118', resources={com1_loc11_out}, trans_max=truck_medium, label='Truck from location 11 to 8', capex=0.0001, vopex=0.05, trans_min=truck_medium)
    
    truck912 = Transport(name='truck912', resources={com1_loc9_out}, trans_max=truck_large, label='Truck from location 9 to 12', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck129 = Transport(name='truck129', resources={com1_loc12_out}, trans_max=truck_large, label='Truck from location 12 to 9', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1012 = Transport(name='truck1012', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 12', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1210 = Transport(name='truck1210', resources={com1_loc12_out}, trans_max=truck_large, label='Truck from location 12 to 10', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1013 = Transport(name='truck1013', resources={com1_loc10_out}, trans_max=truck_large, label='Truck from location 10 to 13', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1310 = Transport(name='truck1310', resources={com1_loc13_out}, trans_max=truck_large, label='Truck from location 13 to 10', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    truck1113 = Transport(name='truck1113', resources={com1_loc11_out}, trans_max=truck_large, label='Truck from location 11 to 13', capex=0.5, vopex=0.05, trans_min=truck_large)
    truck1311 = Transport(name='truck1311', resources={com1_loc13_out}, trans_max=truck_large, label='Truck from location 13 to 11', capex=0.0001, vopex=0.05, trans_min=truck_large)
    
    
    # ======================================================================================================================
    # Declare network
    # ======================================================================================================================
    
    transport_matrix = [
        [[], [], [], [truck14], [truck15], [truck16], [], [], [], [], [], [], []], # source: Location 1
        [[], [], [], [], [truck25], [truck26], [truck27], [], [], [], [], [], []], # source: Location 2
        [[], [], [], [], [], [truck36], [truck37], [truck38], [], [], [], [], []], # source: Location 3
        [[truck41], [], [], [], [], [], [], [], [truck49], [], [], [], []], # source: Location 4
        [[truck51], [truck52], [], [], [], [], [], [], [truck59], [], [], [], []], # source: Location 5
        [[truck61], [truck62], [truck63], [], [], [], [], [], [truck69], [truck610], [], [], []], # source: Location 6
        [[], [truck72], [truck73], [], [], [], [], [], [], [truck710], [truck711], [], []], # source: Location 7
        [[], [], [truck83], [], [], [], [], [], [], [], [truck811], [], []], # source: Location 8
        [[], [], [], [truck94], [truck95], [truck96], [], [], [], [], [], [truck912], []], # source: Location 9
        [[], [], [], [], [], [truck106], [truck107], [], [], [], [], [truck1012], [truck1013]], # source: Location 10
        [[], [], [], [], [], [], [truck117], [truck118], [], [], [], [], [truck1113]], # source: Location 11
        [[], [], [], [], [], [], [], [], [truck129], [truck1210], [], [], []], # source: Location 12
        [[], [], [], [], [], [], [], [], [], [truck1310], [truck1311], [], []] # source: Location 13
    ]
    
    distance_matrix = [
        [0, M, M, 93, 134, 177, M, M, M, M, M, M, M],
        [M, 0, M, M, 120, 163, 106, M, M, M, M, M, M],
        [M, M, 0, M, M, 166, 127, 75, M, M, M, M, M],
        [93, M, M, 0, M, M, M, M, 146, M, M, M, M],
        [134, 120, M, M, 0, M, M, M, 163, M, M, M, M],
        [177, 163, 166, M, M, 0, M, M, 166, 251, M, M, M],
        [M, 106, 127, M, M, M, 0, M, M, 274, 101, M, M],
        [M, M, 75, M, M, M, M, 0, M, M, 139, M, M],
        [M, M, M, 146, 163, 166, M, M, 0, M, M, 234, M],
        [M, M, M, M, M, 251, 274, M, M, 0, M, 296, 294],
        [M, M, M, M, M, M, 101, 139, M, M, 0, M, 219],
        [M, M, M, M, M, M, M, M, 234, 296, M, 0, M],
        [M, M, M, M, M, M, M, M, M, 294, 219, M, 0]
    ]

    locset = [loc1, loc2, loc3, loc4, loc5, loc6, loc7, loc8, loc9, loc10, loc11, loc12, loc13]
    
    sources = locset
    sinks = locset
    
    network = Network(name='Network', scales= scales, source_locations=sources, sink_locations=sinks, transport_matrix=transport_matrix, distance_matrix=distance_matrix, 
      transport_capacity_scale_level=1,
      transport_capacity_factor= {
        (loc1, loc4): {truck14: scen_dict[scen_name]['factor'][['trans14']] if 'trans14' in scen_dict[scen_name]['factor'] else default_df},
        (loc1, loc5): {truck15: scen_dict[scen_name]['factor'][['trans15']] if 'trans15' in scen_dict[scen_name]['factor'] else default_df},
        (loc1, loc6): {truck16: scen_dict[scen_name]['factor'][['trans16']] if 'trans16' in scen_dict[scen_name]['factor'] else default_df},
        (loc2, loc5): {truck14: scen_dict[scen_name]['factor'][['trans25']] if 'trans25' in scen_dict[scen_name]['factor'] else default_df},
        (loc2, loc6): {truck15: scen_dict[scen_name]['factor'][['trans26']] if 'trans26' in scen_dict[scen_name]['factor'] else default_df},
        (loc2, loc7): {truck16: scen_dict[scen_name]['factor'][['trans27']] if 'trans27' in scen_dict[scen_name]['factor'] else default_df},
        (loc3, loc6): {truck36: scen_dict[scen_name]['factor'][['trans36']] if 'trans36' in scen_dict[scen_name]['factor'] else default_df},
        (loc3, loc7): {truck37: scen_dict[scen_name]['factor'][['trans37']] if 'trans37' in scen_dict[scen_name]['factor'] else default_df},
        (loc3, loc8): {truck38: scen_dict[scen_name]['factor'][['trans38']] if 'trans38' in scen_dict[scen_name]['factor'] else default_df},
        (loc4, loc9): {truck49: scen_dict[scen_name]['factor'][['trans49']] if 'trans49' in scen_dict[scen_name]['factor'] else default_df},
        (loc5, loc9): {truck59: scen_dict[scen_name]['factor'][['trans59']] if 'trans59' in scen_dict[scen_name]['factor'] else default_df},
        (loc6, loc9): {truck69: scen_dict[scen_name]['factor'][['trans69']] if 'trans69' in scen_dict[scen_name]['factor'] else default_df},
        (loc6, loc10): {truck610: scen_dict[scen_name]['factor'][['trans610']] if 'trans610' in scen_dict[scen_name]['factor'] else default_df},
        (loc7, loc10): {truck710: scen_dict[scen_name]['factor'][['trans710']] if 'trans710' in scen_dict[scen_name]['factor'] else default_df},
        (loc7, loc11): {truck711: scen_dict[scen_name]['factor'][['trans711']] if 'trans711' in scen_dict[scen_name]['factor'] else default_df},
        (loc8, loc11): {truck811: scen_dict[scen_name]['factor'][['trans811']] if 'trans811' in scen_dict[scen_name]['factor'] else default_df},
        (loc9, loc12): {truck912: scen_dict[scen_name]['factor'][['trans912']] if 'trans912' in scen_dict[scen_name]['factor'] else default_df},
        (loc10, loc12): {truck1012: scen_dict[scen_name]['factor'][['trans1012']] if 'trans1012' in scen_dict[scen_name]['factor'] else default_df},
        (loc10, loc13): {truck1013: scen_dict[scen_name]['factor'][['trans1013']] if 'trans1013' in scen_dict[scen_name]['factor'] else default_df},
        (loc11, loc13): {truck1113: scen_dict[scen_name]['factor'][['trans1113']] if 'trans1113' in scen_dict[scen_name]['factor'] else default_df},
      })
    
    # ======================================================================================================================
    # Declare scenario
    # ======================================================================================================================
    
    daily_demand = {loc12: 250, loc13: 250}
    demand_penalty = {loc12: 50, loc13: 50}
    
    demand_dict = {i: {com1_sold: daily_demand[i]} if i in daily_demand else {com1_sold: 0} for i in locset}
    demand_penalty_dict = {i: {com1_sold: demand_penalty[i]} if i in daily_demand else {com1_sold: 0} for i in locset}
    
    scenario = Scenario(name='scenario_'+scen_name, scales=scales, scheduling_scale_level=2, network_scale_level=0, purchase_scale_level=2, availability_scale_level=1, demand_scale_level=2, capacity_scale_level=1, network=network, demand=demand_dict, demand_penalty=demand_penalty_dict, label='Stochastic scenario with Multiple Locations')
    
    # ======================================================================================================================
    # Declare problem
    # ======================================================================================================================

    problem_mincost = formulate(scenario=scenario,
                            constraints={Constraints.COST, Constraints.TRANSPORT, Constraints.RESOURCE_BALANCE, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.DEMAND, Constraints.NETWORK},
                            demand_sign='eq', objective=Objective.COST_W_DEMAND_PENALTY)

    scale_iter = scale_tuple(instance=problem_mincost, scale_levels=scenario.network_scale_level+1)
    capex_process= sum(problem_mincost.Capex_network[scale_] for scale_ in scale_iter)
    cost_trans_capex = sum(problem_mincost.Capex_transport_network[scale_] for scale_ in scale_iter)

    problem_mincost.first_stage_cost  = capex_process + cost_trans_capex
    
    return scenario, problem_mincost

In [ ]:
scen_cap1_1, model_cap1_1 = build_model(scen_dict=scenario_dictionary, scen_name='cap1_1')

In [ ]:
# scen_probabilities = {
#     'cap1_1': 0.25, 'cap1_2': 0.25, 'cap1_3': 0.25, 'cap1_4': 0.25,                         #Facility Disruption at Location 1
#     'cap2_1': 0.25, 'cap2_2': 0.25, 'cap2_3': 0.25, 'cap2_4': 0.25,                         #Facility Disruption at Location 2
#     'cap3_1': 0.25, 'cap3_2': 0.25, 'cap3_3': 0.25, 'cap3_4': 0.25,                         #Facility Disruption at Location 3
#     'cap4_1': 0.25, 'cap4_2': 0.25, 'cap4_3': 0.25, 'cap4_4': 0.25,                         #Facility Disruption at Location 4
#     'cap5_1': 0.25, 'cap5_2': 0.25, 'cap5_3': 0.25, 'cap5_4': 0.25,                         #Facility Disruption at Location 5
#     'cap6_1': 0.25, 'cap6_2': 0.25, 'cap6_3': 0.25, 'cap6_4': 0.25,                         #Facility Disruption at Location 6
#     'cap7_1': 0.25, 'cap7_2': 0.25, 'cap7_3': 0.25, 'cap7_4': 0.25,                         #Facility Disruption at Location 7
#     'cap8_1': 0.25, 'cap8_2': 0.25, 'cap8_3': 0.25, 'cap8_4': 0.25,                         #Facility Disruption at Location 8
#     'cap9_1': 0.25, 'cap9_2': 0.25, 'cap9_3': 0.25, 'cap9_4': 0.25,                         #Facility Disruption at Location 9
#     'cap10_1': 0.25, 'cap10_2': 0.25, 'cap10_3': 0.25, 'cap10_4': 0.25,                     #Facility Disruption at Location 10
#     'cap11_1': 0.25, 'cap11_2': 0.25, 'cap11_3': 0.25, 'cap11_4': 0.25,                     #Facility Disruption at Location 11
#     'cap12_1': 0.25, 'cap12_2': 0.25, 'cap12_3': 0.25, 'cap12_4': 0.25,                     #Facility Disruption at Location 12
#     'cap13_1': 0.25, 'cap13_2': 0.25, 'cap13_3': 0.25, 'cap13_4': 0.25,                     #Facility Disruption at Location 13
#     'res1_1': 0.25, 'res1_2': 0.25, 'res1_3': 0.25, 'res1_4': 0.25,                         #Resource Disruption at Location 1
#     'res2_1': 0.25, 'res2_2': 0.25, 'res2_3': 0.25, 'res2_4': 0.25,                         #Resource Disruption at Location 2
#     'res3_1': 0.25, 'res3_2': 0.25, 'res3_3': 0.25, 'res3_4': 0.25,                         #Resource Disruption at Location 3
#     'trans14_1': 0.25, 'trans14_2': 0.25, 'trans14_3': 0.25, 'trans14_4': 0.25,             #Link Disruption bw 1 and 4
#     'trans15_1': 0.25, 'trans15_2': 0.25, 'trans15_3': 0.25, 'trans15_4': 0.25,             #Link Disruption bw 1 and 5
#     'trans16_1': 0.25, 'trans16_2': 0.25, 'trans16_3': 0.25, 'trans16_4': 0.25,             #Link Disruption bw 1 and 6
#     'trans25_1': 0.25, 'trans25_2': 0.25, 'trans25_3': 0.25, 'trans25_4': 0.25,             #Link Disruption bw 2 and 5
#     'trans26_1': 0.25, 'trans26_2': 0.25, 'trans26_3': 0.25, 'trans26_4': 0.25,             #Link Disruption bw 2 and 6
#     'trans27_1': 0.25, 'trans27_2': 0.25, 'trans27_3': 0.25, 'trans27_4': 0.25,             #Link Disruption bw 2 and 7
#     'trans36_1': 0.25, 'trans36_2': 0.25, 'trans36_3': 0.25, 'trans36_4': 0.25,             #Link Disruption bw 3 and 6
#     'trans37_1': 0.25, 'trans37_2': 0.25, 'trans37_3': 0.25, 'trans37_4': 0.25,             #Link Disruption bw 3 and 7
#     'trans38_1': 0.25, 'trans38_2': 0.25, 'trans38_3': 0.25, 'trans38_4': 0.25,             #Link Disruption bw 3 and 8
#     'trans49_1': 0.25, 'trans49_2': 0.25, 'trans49_3': 0.25, 'trans49_4': 0.25,             #Link Disruption bw 4 and 9
#     'trans59_1': 0.25, 'trans59_2': 0.25, 'trans59_3': 0.25, 'trans59_4': 0.25,             #Link Disruption bw 5 and 9
#     'trans69_1': 0.25, 'trans69_2': 0.25, 'trans69_3': 0.25, 'trans69_4': 0.25,             #Link Disruption bw 6 and 9
#     'trans610_1': 0.25, 'trans610_2': 0.25, 'trans610_3': 0.25, 'trans610_4': 0.25,         #Link Disruption bw 6 and 10
#     'trans710_1': 0.25, 'trans710_2': 0.25, 'trans710_3': 0.25, 'trans710_4': 0.25,         #Link Disruption bw 7 and 10
#     'trans711_1': 0.25, 'trans711_2': 0.25, 'trans711_3': 0.25, 'trans711_4': 0.25,         #Link Disruption bw 7 and 11
#     'trans811_1': 0.25, 'trans811_2': 0.25, 'trans811_3': 0.25, 'trans811_4': 0.25,         #Link Disruption bw 8 and 11
#     'trans912_1': 0.25, 'trans912_2': 0.25, 'trans912_3': 0.25, 'trans912_4': 0.25,         #Link Disruption bw 9 and 12
#     'trans1012_1': 0.25, 'trans1012_2': 0.25, 'trans1012_3': 0.25, 'trans1012_4': 0.25,     #Link Disruption bw 10 and 12
#     'trans1013_1': 0.25, 'trans1013_2': 0.25, 'trans1013_3': 0.25, 'trans1013_4': 0.25,     #Link Disruption bw 10 and 13
#     'trans1113_1': 0.25, 'trans1113_2': 0.25, 'trans1113_3': 0.25, 'trans1113_4': 0.25      #Link Disruption bw 11 and 13
# }
# 
# all_scenarios = list(scen_probabilities.keys())
# 
# cap_scenarios = [scen for scen in scen_probabilities if 'cap' in scen]
# res_scenarios = [scen for scen in scen_probabilities if 'res' in scen]
# trans_scenarios = [scen for scen in scen_probabilities if 'trans' in scen]
# 
# all_scenarios = all_scenarios + list(itertools.product(cap_scenarios, res_scenarios, trans_scenarios))
# 
# n_dict = {1: 14, 2: 28, 3: 42, 4: 56}
#         
# scen_factors = {scen: [1]*n_dict[int(scen[-1])] + [0]*(_exec_scenarios - n_dict[int(scen[-1])]) for scen in all_scenarios if not isinstance(scen, tuple)}

In [ ]:
# import itertools
# import random
# 
# # Define locations and transport links
# facilities = ['Facility1', 'Facility2', 'Facility3']
# links = ['Link1', 'Link2', 'Link3']
# 
# # Combine all elements into one list
# network_elements = facilities + links
# 
# # Generate all possible disruption states (True = disrupted, False = not disrupted)
# disruption_states = list(itertools.product([True, False], repeat=len(network_elements)))
# 
# # Define an array to store all scenarios
# scenarios = []
# 
# # Generate scenarios
# for state in disruption_states:
#     scenario = {}
#     scenario_prob = 1.0  # Start with a scenario probability of 1
#     
#     for i, element in enumerate(network_elements):
#         # Randomly generate a disruption probability for the element
#         disruption_probability = random.uniform(0, 1)
#         
#         if state[i]:  # If the element is disrupted in this scenario
#             capacity_factor = random.uniform(0, 1)  # Random capacity factor between 0 and 1
#         else:  # If the element is not disrupted
#             capacity_factor = 1
#         
#         # Multiply the scenario probability by the probability of the current state
#         if state[i]:
#             scenario_prob *= disruption_probability  # Multiply by disruption probability
#         else:
#             scenario_prob *= (1 - disruption_probability)  # Multiply by non-disruption probability
#         
#         # Store capacity factor and disruption probability in the scenario
#         scenario[element] = {
#             'capacity_factor': capacity_factor,
#             'disruption_probability': disruption_probability
#         }
#     
#     # Store the overall scenario probability
#     scenario['scenario_probability'] = scenario_prob
#     
#     # Add this scenario to the scenarios array
#     scenarios.append(scenario)
# 
# # Print the resulting scenarios array
# for i, scenario in enumerate(scenarios):
#     print(f"Scenario {i+1}: {scenario}")
#     print('\n')
